In [1]:
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc
from abenc_tg22 import DIPE_ABE, Inner_Product_TG22
from abenc_ph_mj18 import PH_ABE, mat_math, Inner_Product
from nizk import NIZK
from attack import RogueKeyAtt
from collections import defaultdict 
import time
group_tg = PairingGroup('SS512')
group_mj = PairingGroup('MNT159')
assump_size = 2


/home/guy1m0/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [4]:
n = 10
dipe = DIPE_ABE(n, group_tg)
ph_abe = PH_ABE(n, assump_size, group_mj, mat_math())

In [5]:
start_time = time.time()

attributes = Inner_Product_TG22(group_tg)
vec_x,vec_y = attributes.gen_x_y(n)

print ('vec_x:', vec_x)
print ('vec_y:', vec_y)
print ('check x,y:', sum([x*y for x,y in zip(vec_x,vec_y)]))

pp = dipe.setup()
pks, sks = dipe.auth_setup(pp)

GID = group_tg.random(ZR)
M = group_tg.random(GT)

D0, D1, K = dipe.keygen(pp, sks, GID, vec_x)
C = dipe.encrypt(pp, pks, vec_y, M)
M_ = dipe.decrypt(D0, D1, K, C, vec_y)
elapsed_time = time.time() - start_time
if M_ == M:
    print ('Elapsed:', elapsed_time)
else:
    print ('M:',M)
    print ('M_',M_)

vec_x: [150998204040660707077795026364683406150131088534, 201934653701749629896535007478395927196043028086, 189298739501222067770590463315412708008233890265, 720043796071671695857334619612624994435778834549, 478626127974935297319963634805083748606453509702, 202495819437897932526060920400896418572579413302, 262800022846128988203509526595193207521090334846, 325650160003463061856765096121660098315596146213, 391155751084077104935922687592069096817999992971]
vec_y: [1, 1, 1, 1, 1, 1, 1, 1, 1]
check x,y: 0
Elapsed: 0.2512969970703125


In [6]:
attributes = Inner_Product(group_mj)   
nizk = NIZK(group_mj)
att = RogueKeyAtt(n, assump_size)

# sys setup
start_time = time.time()
pp, msk = ph_abe.setup()

# Auth setup
pks, sks = ph_abe.auth_setup(pp)

# Key Gen
GID = group_mj.random(ZR)

vec_x, vec_v = attributes.gen_x_v(n, assump_size)
K, _ = ph_abe.keygen(pp, pks, sks, GID, vec_v)

# Encryption
# start_time = time.time()
M = group_mj.random(GT)
#print ('M:', M)
C, vec_s = ph_abe.encrypt(pp, pks, vec_x, M)

# Decryption
# start_time = time.time()
M_ = ph_abe.decrypt(K, C, vec_v, pp)

elapsed_time = time.time() - start_time
if M_ == M:
    print ('Elapsed:', elapsed_time)
else:
    print ('M:',M)
    print ('M_',M_)



Elapsed: 0.5226294994354248


In [68]:
attributes = Inner_Product(group_mj)   
nizk = NIZK(group_mj)
att = RogueKeyAtt(n, assump_size)
data = {}
data['total']  = 0

# sys setup
start_time = time.time()
pp, msk = ph_abe.setup()
elapsed_time = time.time() - start_time
# data['sys'] = elapsed_time
data['total'] += elapsed_time

# Auth setup
start_time = time.time()
pks, sks = ph_abe.auth_setup(pp)
elapsed_time = time.time() - start_time
# data['auth'] = elapsed_time
data['total'] += elapsed_time

# Key Gen
start_time = time.time()
GID = group_mj.random(ZR)

vec_x, vec_v = attributes.gen_x_v(n, assump_size)
K, _ = ph_abe.keygen(pp, pks, sks, GID, vec_v)
elapsed_time = time.time() - start_time
# data['keygen'] = elapsed_time
data['total'] += elapsed_time

# print ("AA's cost: ", data['auth']/n + elapsed_time/n)

# Worst Case
start_time = time.time()
for i in range(n):
    s_pairs, pis = nizk.prove_pk(pp, pks[str(i+1)], sks[str(i+1)])
elapsed_time = time.time() - start_time
data['prove'] = elapsed_time
#print ("Each cost of proving: ", elapsed_time/n)

start_time = time.time()
for i in range(n):
    check = nizk.verify_pk(pp, s_pairs, pis)
    if not check:
        print ("Check not pass for AA:", str(i+1))
        break
tmp = time.time()
print ("AA's extra cost: ", elapsed_time/n + tmp - start_time)
elapsed_time = tmp - start_time
data['verify'] = elapsed_time
#print ("Verification time: ", elapsed_time)


#print ("user key_gen: ", elapsed_time)

# AD's setup
# start_time = time.time()
ad = vec_v.index(0) + 1
#print ("AD:", ad)
ad_vec_v = [0] * (n-1) + [1]
pks = att.pks_update(ad, pks)
# elapsed_time = time.time() - start_time
# data['ad_setup'] = elapsed_time
#print ("Adv's setup: ", elapsed_time)

# AD KeyGen
#start_time = time.time()
GID_ad = group_mj.random(ZR)
K_,ad_key_gen = ph_abe.keygen(pp, pks, sks, GID_ad, ad_vec_v, ad=ad)
#print ("For ad with index: ", ad, "takes :", ad_key_gen)

#elapsed_time = time.time() - start_time
data['ad_keygen'] = ad_key_gen
#print ("Ad's cost: ", data['ad_setup'] + )

# Encryption
# start_time = time.time()
M = group_mj.random(GT)
#print ('M:', M)
C, vec_s = ph_abe.encrypt(pp, pks, vec_x, M)
# elapsed_time = time.time() - start_time
# print ("DO's cost: ", elapsed_time)
# data['encrypt'] = elapsed_time
data['total'] += elapsed_time

# Decryption
# start_time = time.time()
M_ = ph_abe.decrypt(K, C, vec_v, pp)
if M_ != M:
    print ('Error in decrypt M (usr): ', M_)
# elapsed_time = time.time() - start_time
# print ("DU's cost: ", elapsed_time)
# data['decrypt'] = elapsed_time
data['total'] += elapsed_time

# AD decrypt
tmp = ph_abe.decrypt(K_, C, ad_vec_v, pp)

# start_time = time.time()
omega = att.gen_omega(K_,C)
M_ =  tmp * mat_math().prod(omega)
if M_ != M:
    print ('Error in decrypt M (adv): ', M_)
# elapsed_time = time.time() - start_time
# data['ad_cancel_out'] = elapsed_time
# print ("Ad's cost: ", data['ad_setup'] + data['ad_keygen'] + data['ad_cancel_out'])

print ("Original Sys cost: ", data['total'])
print ("Enhanced Sys cost: ", data['total'] + data['prove'] + data['verify'])


AA's extra cost:  0.11915111541748047
Original Sys cost:  0.46268200874328613
Enhanced Sys cost:  0.629896879196167


In [64]:
group_mj.random(ZR)

72875351977953222903237743599457076121983599353

In [65]:
group_tg.random(ZR)

111346151623188425218217829626386290813257792045

True